In [1]:
#encoding=utf-8
import json
import os
import os.path

In [2]:
ycj_news_path = "/home/xiangzi777/workspace/python/Scrapy/yuncaijing/ycj_news"
#ycj_news_path = "/home/xiangzi777/workspace/python/DM/theme_description/Data/crawl_data/theme_name/ycj_news"

In [3]:
#encoding=utf-8
import json
import os
import os.path
ycj_news_path = "/home/xiangzi777/workspace/python/Scrapy/yuncaijing/ycj_news"
#ycj_news_path = "/home/xiangzi777/workspace/python/DM/theme_description/Data/crawl_data/theme_name/ycj_news"
ycj_news_file_name_list = [os.path.join(root, name) for root, dirs, files in os.walk(ycj_news_path) for name in files]        

In [4]:
#主题关系权重字典： '(A, B) : weight'
import pickle
def get_theme_edge_list(ycj_news_file_name_list):
    theme_edge_dict = {}
    theme_set = set()
    theme_edge_list = []
    #主题:主题表字典： {A:[B, C], B:[A, D], ...}
    theme_theme_edge_dict = {}
    for file_path in ycj_news_file_name_list[0:30000]:
        with open(file_path) as news_json_file:
            #print file_path
            try:
                json_data = json.load(news_json_file)
            except:
                print file_path
            related_themes =  json_data['related_theme'].strip().split(' ')
            cur_theme_set = set(related_themes)
            theme_set |= cur_theme_set
        #theme_edge_list = [(themeA, themeB) for themeA in related_themes, ]
            theme_edge_list = [(related_themes[i], related_themes[j]) for i in range(0, len(related_themes)) for j in range(i+1, len(related_themes))]
            for theme_edge in theme_edge_list:
                if theme_edge not in theme_edge_dict:
                    theme_edge_dict[theme_edge] = 0
                theme_edge_dict[theme_edge] += 1
                theme_theme_edge_dict[theme_edge[0]] = set()
                theme_theme_edge_dict[theme_edge[1]] = set()
    #theme_edge_list = [(theme_edge[0], theme_edge[1]) for theme_edge in theme_edge_dict]
    for theme_edge in theme_edge_dict:
        #if (u"P2P" in theme_edge[0] or u"P2P" in theme_edge[1]) and theme_edge_dict[theme_edge] > 5:
        if theme_edge_dict[theme_edge] > 3:
            #print theme_edge[0], theme_edge[1]
            theme_edge_list.append(theme_edge)
            theme_theme_edge_dict[theme_edge[0]].add(theme_edge)
            theme_theme_edge_dict[theme_edge[1]].add(theme_edge)
    #pickle.dump(theme_edge_list, open('theme_edge_list.pickle', 'w'))
    return theme_edge_list, theme_theme_edge_dict, theme_edge_dict
theme_edge_list, theme_theme_edge_dict, theme_edge_weight_dict = get_theme_edge_list(ycj_news_file_name_list)

/home/xiangzi777/workspace/python/Scrapy/yuncaijing/ycj_news/.2692388.ycj.swp


In [5]:
import pickle
pickle.dump(theme_theme_edge_dict, open('theme_theme_edge_dict.pickle', 'w'))
pickle.dump(theme_theme_edge_dict, open('theme_edge_dict.pickle', 'w'))

In [ ]:
print theme_edge_list

In [6]:
import networkx as NX
import matplotlib.pyplot as PLT
import matplotlib
Gh = NX.Graph()
#Gh.add_edge(u"中文", "Bike", weight=1.0)
#Gh.add_edge("Bike", "Apple", weight=0.9)
#Gh.add_edge("Me", "Bike", weight=1.1)
Gh.add_edges_from(theme_edge_list)
all_nodes = Gh.nodes()
#print type(Gh.edges())
scaled_node_size = lambda(node) : NX.degree(Gh, node) * 200
print type(scaled_node_size)
#position = NX.spring_layout(Gh)    # just choose a layout scheme
#PLT.figure(figsize=(24, 12), dpi=80)
#NX.draw_networkx_nodes(Gh, position, node_size=map(scaled_node_size, all_nodes), font_family='sans-serif')
#NX.draw_networkx_labels(Gh,position, font_family='SimHei')
#NX.draw_networkx_edges(Gh, position, Gh.edges(), width=0.5, alpha=0.9, edge_color="green", font_family='sans-serif')
#dots per inch
#PLT.axis("off")
#PLT.show()
#PLT.savefig("test.png")

<type 'function'>


In [ ]:
#degree = Gh.degree()

In [7]:
#获得度大于5的节点：good_theme_name_list
degree = Gh.degree()
#good_theme_name_list = []
#for key in degree:
#    if degree[key] > 5:
#        good_theme_name_list.append((key, degree[key]))
        #print key, degree[key]
#degree = sorted(degree.values())
#print degree

In [8]:
cur_edge_list = list(theme_theme_edge_dict[u'互联网+'])

In [9]:
#根据节点的边和节点的度绘制关系图
def plot_theme_graph(cur_theme_degree, cur_edge_list, edge_color_list, degree_dict, does_show):
    PLT.figure(figsize=(24, 12), dpi=80)
    G = NX.Graph()
    G.add_edges_from(cur_edge_list)
    pos=NX.spring_layout(G)
    
    edge_colors=edge_color_list
    #edge_colors=range(G.size())
    #node_colors=range(G.number_of_nodes())
    node_colors = [degree_dict[node] for node in G.nodes()]
    scaled_node_size = lambda(node) : degree_dict[node] * 20
    #node_size_list = 30 * node_colors
    #node_colors='#A0CBE2'
    #style='dashed|solid', font_size=12, node_size=3000, node_color='#A0CBE2',
    NX.draw(G,pos, node_color=node_colors, node_size = map(scaled_node_size, G.nodes()),edge_color=edge_colors,width=4,cmap=PLT.cm.Reds,edge_cmap=PLT.cm.Blues,edge_vmin = -10, edge_vmax = 30, with_labels=True)
    PLT.savefig("./theme_graph/" + cur_theme_degree[0] + str(cur_theme_degree[1]) + ".png") # save as png
    if does_show: PLT.show() # display

In [10]:
#获得一条边上，除了主节点的另一个节点
def get_other_node_on_edge(edge, cur_theme_name):
    if edge[0] == cur_theme_name:
        return edge[1]
    elif edge[1] == cur_theme_name:
        return edge[0]
    else:
        return None

In [11]:
#绘制所有满足度大于degree_filter_flag的节点的关系图；
def plot_all_good_theme_graph(degree_dict, theme_theme_edge_dict, degree_filter_flag):
    good_theme_name_list = [(key, degree_dict[key]) for key in degree_dict if degree_dict[key] > degree_filter_flag]
    for theme_name_degree in good_theme_name_list:
        plot_single_theme_graph(theme_name_degree, degree_dict, theme_theme_edge_dict, False)
#绘制一个节点的关系图；
def plot_single_theme_graph(theme_name_degree, degree_dict, theme_theme_edge_dict, does_show=False):
        theme_name = theme_name_degree[0]
        #print theme_name
        #theme_degree = theme_name_degree[1]
        if theme_name not in theme_theme_edge_dict:
            return "None"
        cur_edge_set = theme_theme_edge_dict[theme_name]
        other_node_list = [get_other_node_on_edge(edge, theme_name) for edge in cur_edge_set]
        #print other_node_list
        extend_theme_node_list = [node for node in other_node_list if degree_dict[node] <= 4]
        #print extend_theme_node_list
        #print extend_theme_node_list
        extend_edge_set_list = [theme_theme_edge_dict[extend_theme] for extend_theme in extend_theme_node_list]
        extend_edge_set = set().union(*extend_edge_set_list)
        #print extend_edge_set
        edge_list = list(cur_edge_set | extend_edge_set)
        
        edge_color_list = [theme_edge_weight_dict[theme_edge] for theme_edge in edge_list]
        #node_color_list = []
        #node_size_list = [degree_dict[theme_name] for theme_name in ]
        plot_theme_graph(theme_name_degree, edge_list, edge_color_list, degree_dict, does_show)

In [12]:
plot_single_theme_graph((u'国企改革', 60), degree, theme_theme_edge_dict, True)

/usr/lib/pymodules/python2.7/matplotlib/collections.py:507: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors_original != 'face':
/usr/lib/pymodules/python2.7/matplotlib/collections.py:446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [9]:
plot_all_good_theme_graph(degree, theme_theme_edge_dict, 7)

In [13]:
#绘制themes_lv1.txt中的主题关系图；并获得缺失的主题；
theme_lv1_list = open('themes_lv1.txt').read().split('\n')
absent_theme_list = []
for theme in theme_lv1_list:
    if "None" == plot_single_theme_graph((theme.decode('utf-8'), 30), degree, theme_theme_edge_dict, False):
        absent_theme_list.append(theme)

In [14]:
print len(absent_theme_list)
for theme in absent_theme_list:
    print theme

16
自贸区
重组
高送转
二胎
ST
苹果
信息安全
虚拟现实
举牌
中字头
依法治国
环境保护
水利建设
交通运输
次新



In [ ]:
NX.clustering(Gh)

In [ ]:
component_num = NX.number_connected_components(Gh)
print component_num

In [ ]:
G=NX.star_graph(20)
pos=NX.spring_layout(G)
colors=range(20)
node_colors=range(21)
#node_color='#A0CBE2'
NX.draw(G,pos,node_color=node_colors, edge_color=colors,width=4,edge_cmap=PLT.cm.Blues,with_labels=True)
PLT.savefig("edge_colormap.png") # save as png
PLT.show() # display